In [1]:
# !pip install -r requirements.txt
# from bertviz import head_view
# from transformers import BertTokenizer, BertModel

In [2]:
%%javascript
require.config({
  paths: {
      d3: '//cdnjs.cloudflare.com/ajax/libs/d3/3.4.8/d3.min',
      jquery: '//ajax.googleapis.com/ajax/libs/jquery/2.0.0/jquery.min',
  }
});

<IPython.core.display.Javascript object>

In [3]:
# def show_head_view(model, tokenizer, sentence_a, sentence_b=None):
#     inputs = tokenizer.encode_plus(sentence_a, sentence_b, return_tensors='pt', add_special_tokens=True)
#     input_ids = inputs['input_ids']
#     if sentence_b:
#         token_type_ids = inputs['token_type_ids']
#         attention = model(input_ids, token_type_ids=token_type_ids)[-1]
#         sentence_b_start = token_type_ids[0].tolist().index(1)
#     else:
#         attention = model(input_ids)[-1]
#         sentence_b_start = None
#     input_id_list = input_ids[0].tolist() # Batch index 0
#     tokens = tokenizer.convert_ids_to_tokens(input_id_list)    
#     head_view(attention, tokens, sentence_b_start)

In [4]:
# model_version = 'bert-base-uncased'
# do_lower_case = True
# model = BertModel.from_pretrained(model_version, output_attentions=True)
# tokenizer = BertTokenizer.from_pretrained(model_version, do_lower_case=do_lower_case)
# sentence_a = "The cat sat on the mat"
# sentence_b = "The cat lay on the rug"
# show_head_view(model, tokenizer, sentence_a, sentence_b)

In [46]:
from bertviz import head_view
import torch
import pickle
from bertviz import head_view
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, BertConfig,AutoTokenizer
import csv

# Set parameters for loading file
ERROR_OR_CORRECT = "error"
POSITION = "peer"
USE_TAG = "rel_posi"
BATCH_SIZE = 16
EPOCH = 2

# Load the set of index, pred_label, groundtruth, utterance.
csv_for_listing_PATH = "/Users/natsuki/Documents/University_Taiwan/M1_Spring/LAB/dataset/{}_set/{}_{}_{}_b_{}_e_{}.csv".format(ERROR_OR_CORRECT,ERROR_OR_CORRECT,POSITION,USE_TAG,BATCH_SIZE,EPOCH)
# csv_for_listing_PATH = "/Users/natsuki/Documents/University_Taiwan/M1_Spring/LAB/dataset/{}_set/{}_{}_b_{}_e_{}.csv".format(ERROR_OR_CORRECT,ERROR_OR_CORRECT,USE_TAG,BATCH_SIZE,EPOCH)
with open(csv_for_listing_PATH, 'r') as f:
    reader = csv.reader(f)
    output_set = [row for row in reader]
# print("len: {}".format(len(output_set)))
# print([x for x in output_set])
# print(output_set[0][3][:10])

# # # Fistering
filtered = []
for o in output_set:
    if "'爸', '，', '我'" in o[3]:
        filtered.append(o)
print("len: {}".format(len(filtered)))
print([x for x in filtered])


# Load utterance
# INDEX= int(output_set[7][0])
# pkl_f_name = "utterance/{}_b_{}_e_{}_i_{}".format(USE_TAG, BATCH_SIZE, EPOCH ,INDEX)
# pkl_path = "/Users/natsuki/Documents/University_Taiwan/M1_Spring/LAB/dataset/pkl/{}".format(pkl_f_name)
# with open(pkl_path, 'rb') as f:
#     utterance = pickle.load(f)
# print(utterance)

#  ['1483', 'child', 'child', '爸，我在北京就是擔心你們祖孫三人，今天趁學校放小假，我特意趕回來看看你們。'], ['1507', 'child', 'child', '爸，我爲什麼要去考博不就是爲了她，爲了我們這個家嗎！'], ['2016', 'child', 'child', '爸，我也十分理解你們老人的心情，生病時盼望兒女們在自己身邊，這也是人之常情，但是，我們現在都在上班，各有各的事情，要想天天守在您們身邊也是不可能的事，前次左小龍父親提出，要求我倆調到一起去工作，這也是考慮照顧雙方老人而起見的。我倆誰調動，我們正在商量，可能我的調動性要大些。']

len: 2
[['1278', 'peer', 'inferior', "['[CLS]', '爸', '爸', '，', '我', '在', '這', '裏', '呢', '！', '你', '怎', '麼', '不', '看', '見', '，', '簡', '直', '是', '瞎', '問', '！', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD

In [48]:
# Set parameters for loading file
# INDEX = output_set[0][0]
INDEX = filtered[1][0]

# Define pretrained model and tokenizer
PRETRAINED_MODEL_NAME = "bert-base-chinese" 
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)

# Load attentions
pkl_f_name = "attention/{}_b_{}_e_{}_i_{}".format(USE_TAG, BATCH_SIZE, EPOCH ,INDEX)
pkl_path = "/Users/natsuki/Documents/University_Taiwan/M1_Spring/LAB/dataset/pkl/{}".format(pkl_f_name)
with open(pkl_path, 'rb') as f:
    attentions = pickle.load(f)

# Load utterance
pkl_f_name = "utterance/{}_b_{}_e_{}_i_{}".format(USE_TAG, BATCH_SIZE, EPOCH ,INDEX)
pkl_path = "/Users/natsuki/Documents/University_Taiwan/M1_Spring/LAB/dataset/pkl/{}".format(pkl_f_name)
with open(pkl_path, 'rb') as f:
    utterance = pickle.load(f)

# Cut padding
for index,u in enumerate(utterance):
    if u == '[SEP]':
        end = index
        break   
utterance = utterance[:end]
attentions=torch.stack(attentions)
attentions=attentions[:,:,:,:end,:end]

# Return the format of attentions to original 
temp = []
for a in attentions:
    temp.append(a)
attentions = tuple(temp)

# Show the attentions
head_view(attentions, utterance, None)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>